# Python heapq 模块详解

heapq 是 Python 标准库中的堆队列（优先队列）实现模块。它提供了基于列表的最小堆实现。

## 什么是堆？

堆（Heap）是一种特殊的完全二叉树数据结构，具有以下特点：

### 1. 结构特性
- **完全二叉树**：除了最后一层，其他层都被完全填满，最后一层从左到右填充
- **数组存储**：可以用数组（列表）来表示，不需要额外的指针
- **父子关系**：对于数组索引 i 的节点：
  - 左子节点索引：`2*i + 1`
  - 右子节点索引：`2*i + 2`
  - 父节点索引：`(i-1)//2`

### 2. 堆性质（Heap Property）
- **最小堆**：父节点的值总是 ≤ 其子节点的值（堆顶是最小值）
- **最大堆**：父节点的值总是 ≥ 其子节点的值（堆顶是最大值）

### 3. 堆的可视化表示

```
最小堆示例：[1, 3, 2, 4, 6, 8, 5, 9, 7]

数组形式：[1, 3, 2, 4, 6, 8, 5, 9, 7]
索引：    [0, 1, 2, 3, 4, 5, 6, 7, 8]

树形结构：
         1 (索引0)
       /   \
      3     2 (索引1,2)
     / \   / \
    4   6 8   5 (索引3,4,5,6)
   / \
  9   7 (索引7,8)
```

### 4. 为什么使用堆？

堆的主要优势：
- **高效的最值访问**：O(1) 时间获取最小/最大值
- **高效的插入删除**：O(log n) 时间复杂度
- **空间效率**：直接使用数组存储，无需额外指针
- **部分排序**：无需完全排序就能快速找到前K个最值

## Python heapq 的实现特点

Python 的 heapq 模块：
- 实现的是**最小堆**
- 直接在普通 Python 列表上操作
- 使用零索引（第一个元素索引为 0）
- 提供了丰富的堆操作函数

## 时间复杂度总览
- **插入元素（heappush）**：O(log n)
- **删除最小元素（heappop）**：O(log n)
- **查看最小元素**：O(1) - 直接访问 heap[0]
- **建堆（heapify）**：O(n)
- **查找前K个元素（nlargest/nsmallest）**：O(n + k log n)

In [1]:
import heapq
import random

# 导入模块
print("heapq 模块已导入")

heapq 模块已导入


### 4. heapify() - 建堆操作详解

#### 算法原理
heapify 操作将一个无序数组转换为堆，使用"自底向上"的方法，从最后一个非叶子节点开始，依次执行下沉操作。

#### 为什么 heapify 是 O(n) 而不是 O(n log n)？

这是一个常见的误解。虽然每次下沉操作是 O(log n)，但 heapify 的总体复杂度是 O(n)。

**关键洞察**：
- 大部分节点都在树的底层，下沉距离很短
- 只有少数节点需要下沉很长距离
- 最后一层节点（约 n/2 个）根本不需要下沉

#### 数学分析
对于一个有 n 个节点的完全二叉树：
- 高度为 h 的层最多有 2^h 个节点
- 这些节点最多下沉 (log n - h) 层

总操作次数：
```
T(n) = Σ(h=0 to log n) 2^h × (log n - h)
     = n × Σ(h=0 to log n) (log n - h) / 2^(log n - h)
     ≤ n × Σ(k=1 to ∞) k / 2^k
     = n × 2  # 这是一个收敛的几何级数
     = O(n)
```

#### heapify 伪代码
```python
def heapify_algorithm(array):
    n = len(array)
    
    # 从最后一个非叶子节点开始
    # 最后一个非叶子节点的索引是 (n-1-1)//2 = (n-2)//2
    for i in range((n - 2) // 2, -1, -1):
        sift_down(array, i, n)

def sift_down(heap, parent_index, heap_size):
    while True:
        left_child = 2 * parent_index + 1
        right_child = 2 * parent_index + 2
        smallest = parent_index
        
        if (left_child < heap_size and 
            heap[left_child] < heap[smallest]):
            smallest = left_child
            
        if (right_child < heap_size and 
            heap[right_child] < heap[smallest]):
            smallest = right_child
        
        if smallest == parent_index:
            break
            
        heap[parent_index], heap[smallest] = heap[smallest], heap[parent_index]
        parent_index = smallest
```

### 5. 堆的应用场景和优势

#### 适用场景
1. **优先队列**：任务调度，事件处理
2. **Top K 问题**：找出最大/最小的 K 个元素
3. **流数据处理**：实时维护数据的中位数
4. **图算法**：Dijkstra 最短路径算法
5. **合并操作**：合并多个有序序列

#### 与其他数据结构的对比

| 操作 | 堆 | 数组（无序） | 数组（有序） | 平衡BST |
|------|-----|-------------|-------------|---------|
| 插入 | O(log n) | O(1) | O(n) | O(log n) |
| 删除最值 | O(log n) | O(n) | O(1) | O(log n) |
| 查找最值 | O(1) | O(n) | O(1) | O(log n) |
| 查找任意元素 | O(n) | O(n) | O(log n) | O(log n) |
| 空间复杂度 | O(1) 额外 | O(1) | O(1) | O(n) 指针 |

**堆的优势**：
- 在数组基础上实现，空间效率高
- 插入和删除最值都是 O(log n)
- 建堆操作是 O(n)，比排序更快
- 不需要维护全局排序，只关心最值

In [2]:
# 演示堆操作的详细过程

def visualize_heap_operations():
    """可视化堆操作过程"""
    
    print("=== 堆插入操作演示 ===")
    print("我们将演示向空堆中依次插入元素：3, 1, 4, 1, 5")
    print()
    
    # 自定义的堆实现，用于演示
    class DemoHeap:
        def __init__(self):
            self.heap = []
            
        def push(self, item):
            print(f"插入元素 {item}:")
            self.heap.append(item)
            print(f"  1. 将 {item} 添加到末尾: {self.heap}")
            
            # 上浮操作
            child_idx = len(self.heap) - 1
            steps = 0
            
            while child_idx > 0:
                parent_idx = (child_idx - 1) // 2
                steps += 1
                
                print(f"  2.{steps} 比较子节点 heap[{child_idx}]={self.heap[child_idx]} 与父节点 heap[{parent_idx}]={self.heap[parent_idx]}")
                
                if self.heap[child_idx] >= self.heap[parent_idx]:
                    print(f"     -> 子节点 >= 父节点，堆性质满足，停止上浮")
                    break
                    
                print(f"     -> 子节点 < 父节点，交换位置")
                self.heap[child_idx], self.heap[parent_idx] = self.heap[parent_idx], self.heap[child_idx]
                print(f"     -> 交换后: {self.heap}")
                
                child_idx = parent_idx
            
            print(f"  最终堆: {self.heap}")
            print(f"  树形结构: {self.get_tree_representation()}")
            print()
            
        def pop(self):
            if not self.heap:
                return None
                
            print(f"删除堆顶元素:")
            print(f"  1. 当前堆: {self.heap}")
            
            min_val = self.heap[0]
            print(f"  2. 保存最小值: {min_val}")
            
            # 将最后一个元素移到堆顶
            last_element = self.heap.pop()
            print(f"  3. 取出最后一个元素: {last_element}")
            
            if self.heap:
                self.heap[0] = last_element
                print(f"  4. 将最后元素放到堆顶: {self.heap}")
                
                # 下沉操作
                parent_idx = 0
                steps = 0
                
                while True:
                    left_child = 2 * parent_idx + 1
                    right_child = 2 * parent_idx + 2
                    smallest = parent_idx
                    steps += 1
                    
                    print(f"  5.{steps} 检查节点 heap[{parent_idx}]={self.heap[parent_idx]} 与其子节点")
                    
                    # 找最小的子节点
                    if (left_child < len(self.heap) and 
                        self.heap[left_child] < self.heap[smallest]):
                        smallest = left_child
                        print(f"      -> 左子节点 heap[{left_child}]={self.heap[left_child]} 更小")
                        
                    if (right_child < len(self.heap) and 
                        self.heap[right_child] < self.heap[smallest]):
                        smallest = right_child
                        print(f"      -> 右子节点 heap[{right_child}]={self.heap[right_child]} 更小")
                    
                    if smallest == parent_idx:
                        print(f"      -> 父节点已是最小，堆性质满足，停止下沉")
                        break
                        
                    print(f"      -> 交换父节点 heap[{parent_idx}] 与子节点 heap[{smallest}]")
                    self.heap[parent_idx], self.heap[smallest] = self.heap[smallest], self.heap[parent_idx]
                    print(f"      -> 交换后: {self.heap}")
                    
                    parent_idx = smallest
            
            print(f"  最终堆: {self.heap}")
            print(f"  返回值: {min_val}")
            print()
            return min_val
            
        def get_tree_representation(self):
            if not self.heap:
                return "空堆"
            
            # 简单的树形表示
            if len(self.heap) == 1:
                return f"{self.heap[0]}"
            elif len(self.heap) <= 3:
                result = f"    {self.heap[0]}\\n"
                if len(self.heap) > 1:
                    result += f"   / "
                    if len(self.heap) > 2:
                        result += f"\\\\\\n  {self.heap[1]}   {self.heap[2]}"
                    else:
                        result += f"\\n  {self.heap[1]}"
                return result
            else:
                # 对于更复杂的堆，只显示前几层
                return f"根节点: {self.heap[0]}, 第二层: {self.heap[1:3]}, 第三层: {self.heap[3:7] if len(self.heap) > 3 else '[]'}"
    
    # 演示插入操作
    demo_heap = DemoHeap()
    for value in [3, 1, 4, 1, 5]:
        demo_heap.push(value)
    
    print("\\n=== 堆删除操作演示 ===")
    print("现在我们从堆中删除元素:")
    print()
    
    # 演示删除操作
    while demo_heap.heap:
        demo_heap.pop()

# 运行演示
visualize_heap_operations()

=== 堆插入操作演示 ===
我们将演示向空堆中依次插入元素：3, 1, 4, 1, 5

插入元素 3:
  1. 将 3 添加到末尾: [3]
  最终堆: [3]
  树形结构: 3

插入元素 1:
  1. 将 1 添加到末尾: [3, 1]
  2.1 比较子节点 heap[1]=1 与父节点 heap[0]=3
     -> 子节点 < 父节点，交换位置
     -> 交换后: [1, 3]
  最终堆: [1, 3]
  树形结构:     1\n   / \n  3

插入元素 4:
  1. 将 4 添加到末尾: [1, 3, 4]
  2.1 比较子节点 heap[2]=4 与父节点 heap[0]=1
     -> 子节点 >= 父节点，堆性质满足，停止上浮
  最终堆: [1, 3, 4]
  树形结构:     1\n   / \\\n  3   4

插入元素 1:
  1. 将 1 添加到末尾: [1, 3, 4, 1]
  2.1 比较子节点 heap[3]=1 与父节点 heap[1]=3
     -> 子节点 < 父节点，交换位置
     -> 交换后: [1, 1, 4, 3]
  2.2 比较子节点 heap[1]=1 与父节点 heap[0]=1
     -> 子节点 >= 父节点，堆性质满足，停止上浮
  最终堆: [1, 1, 4, 3]
  树形结构: 根节点: 1, 第二层: [1, 4], 第三层: [3]

插入元素 5:
  1. 将 5 添加到末尾: [1, 1, 4, 3, 5]
  2.1 比较子节点 heap[4]=5 与父节点 heap[1]=1
     -> 子节点 >= 父节点，堆性质满足，停止上浮
  最终堆: [1, 1, 4, 3, 5]
  树形结构: 根节点: 1, 第二层: [1, 4], 第三层: [3, 5]

\n=== 堆删除操作演示 ===
现在我们从堆中删除元素:

删除堆顶元素:
  1. 当前堆: [1, 1, 4, 3, 5]
  2. 保存最小值: 1
  3. 取出最后一个元素: 5
  4. 将最后元素放到堆顶: [5, 1, 4, 3]
  5.1 检查节点 heap[0]=5 与其子节点
      -> 左子节点 heap[1]=1 更小

## 1. heappush() - 向堆中添加元素

将元素添加到堆中，并保持堆的性质。

## 堆操作的详细算法分析

在深入学习 heapq 的具体函数之前，让我们先理解堆的核心操作原理和时间复杂度分析。

### 1. heappush() - 插入操作详解

#### 算法原理
插入新元素到堆中需要维护堆的性质，主要通过"上浮"（sift up）操作实现。

#### 算法步骤
1. 将新元素添加到堆的末尾（数组最后一个位置）
2. 比较新元素与其父节点
3. 如果新元素小于父节点（最小堆），则交换
4. 重复步骤2-3，直到满足堆性质或到达根节点

#### 伪代码
```python
def heappush_algorithm(heap, item):
    # 1. 将新元素添加到末尾
    heap.append(item)
    
    # 2. 上浮操作（sift up）
    child_index = len(heap) - 1  # 新元素的索引
    
    while child_index > 0:
        parent_index = (child_index - 1) // 2
        
        # 如果子节点 >= 父节点，堆性质已满足
        if heap[child_index] >= heap[parent_index]:
            break
            
        # 交换父子节点
        heap[child_index], heap[parent_index] = heap[parent_index], heap[child_index]
        
        # 继续向上检查
        child_index = parent_index
```

#### 时间复杂度分析
- **最好情况**：O(1) - 新元素正好比父节点大，无需交换
- **最坏情况**：O(log n) - 新元素需要一直上浮到根节点
- **平均情况**：O(log n) - 期望上浮高度约为树高的一半

**为什么是 O(log n)？**
- 完全二叉树的高度为 log₂(n)
- 上浮操作最多执行树的高度次
- 每次比较和交换都是 O(1) 操作

### 2. heappop() - 删除操作详解

#### 算法原理
删除堆顶元素后需要重新调整堆，主要通过"下沉"（sift down）操作实现。

#### 算法步骤
1. 保存堆顶元素（要返回的最小值）
2. 将堆的最后一个元素移到堆顶
3. 删除最后一个元素
4. 从堆顶开始执行下沉操作，恢复堆性质

#### 伪代码
```python
def heappop_algorithm(heap):
    if not heap:
        raise IndexError("heap is empty")
    
    # 1. 保存要返回的最小值
    min_value = heap[0]
    
    # 2. 将最后一个元素移到堆顶
    last_element = heap.pop()  # 同时删除最后一个元素
    
    if heap:  # 如果堆不为空
        heap[0] = last_element
        
        # 3. 下沉操作（sift down）
        parent_index = 0
        heap_size = len(heap)
        
        while True:
            left_child = 2 * parent_index + 1
            right_child = 2 * parent_index + 2
            smallest = parent_index
            
            # 找到父节点和两个子节点中的最小值
            if (left_child < heap_size and 
                heap[left_child] < heap[smallest]):
                smallest = left_child
                
            if (right_child < heap_size and 
                heap[right_child] < heap[smallest]):
                smallest = right_child
            
            # 如果父节点已经是最小的，堆性质满足
            if smallest == parent_index:
                break
                
            # 交换父节点与最小子节点
            heap[parent_index], heap[smallest] = heap[smallest], heap[parent_index]
            
            # 继续向下检查
            parent_index = smallest
    
    return min_value
```

#### 时间复杂度分析
- **最好情况**：O(1) - 新的堆顶元素正好比子节点小
- **最坏情况**：O(log n) - 元素需要一直下沉到叶子节点
- **平均情况**：O(log n) - 期望下沉高度约为树高的一半

**为什么是 O(log n)？**
- 下沉操作最多执行树的高度次（log₂(n)）
- 每次需要比较最多3个节点（父节点和两个子节点）
- 每次比较和交换都是 O(1) 操作

### 3. 图解示例：插入操作

让我们用一个具体例子来理解插入操作：

In [3]:
# 创建一个空列表作为堆
heap = []

# 向堆中添加元素
heapq.heappush(heap, 3)
heapq.heappush(heap, 1)
heapq.heappush(heap, 4)
heapq.heappush(heap, 1)
heapq.heappush(heap, 5)

print(f"堆的内容: {heap}")
print(f"最小元素: {heap[0]}")

堆的内容: [1, 1, 4, 3, 5]
最小元素: 1


## 2. heappop() - 弹出并返回最小元素

删除并返回堆中的最小元素，同时保持堆的性质。

In [4]:
# 从堆中弹出最小元素
min_element = heapq.heappop(heap)
print(f"弹出的最小元素: {min_element}")
print(f"弹出后的堆: {heap}")

# 继续弹出
while heap:
    print(f"弹出: {heapq.heappop(heap)}, 剩余堆: {heap}")

弹出的最小元素: 1
弹出后的堆: [1, 3, 4, 5]
弹出: 1, 剩余堆: [3, 5, 4]
弹出: 3, 剩余堆: [4, 5]
弹出: 4, 剩余堆: [5]
弹出: 5, 剩余堆: []


## 3. heappushpop() - 推入并弹出

先向堆中推入一个元素，然后弹出并返回最小元素。这比分别调用 heappush() 和 heappop() 更高效。

In [5]:
# 重新创建堆
heap = [1, 3, 5, 7, 9]
heapq.heapify(heap)
print(f"原始堆: {heap}")

# 推入 2 并弹出最小值
result = heapq.heappushpop(heap, 2)
print(f"推入 2 并弹出最小值: {result}")
print(f"操作后的堆: {heap}")

# 推入 0 并弹出最小值
result = heapq.heappushpop(heap, 0)
print(f"推入 0 并弹出最小值: {result}")
print(f"操作后的堆: {heap}")

原始堆: [1, 3, 5, 7, 9]
推入 2 并弹出最小值: 1
操作后的堆: [2, 3, 5, 7, 9]
推入 0 并弹出最小值: 0
操作后的堆: [2, 3, 5, 7, 9]


## 4. heapreplace() - 弹出并推入

先弹出并返回最小元素，然后推入一个新元素。

In [6]:
# 重新创建堆
heap = [1, 3, 5, 7, 9]
heapq.heapify(heap)
print(f"原始堆: {heap}")

# 弹出最小值并推入 4
result = heapq.heapreplace(heap, 4)
print(f"弹出最小值并推入 4, 弹出的值: {result}")
print(f"操作后的堆: {heap}")

# 弹出最小值并推入 0
result = heapq.heapreplace(heap, 0)
print(f"弹出最小值并推入 0, 弹出的值: {result}")
print(f"操作后的堆: {heap}")

原始堆: [1, 3, 5, 7, 9]
弹出最小值并推入 4, 弹出的值: 1
操作后的堆: [3, 4, 5, 7, 9]
弹出最小值并推入 0, 弹出的值: 3
操作后的堆: [0, 4, 5, 7, 9]


## 5. heapify() - 将列表转换为堆

将任意列表就地转换为堆结构。

In [7]:
# 创建一个无序列表
data = [9, 3, 8, 1, 6, 2, 5, 4, 7]
print(f"原始列表: {data}")

# 将列表转换为堆
heapq.heapify(data)
print(f"转换为堆后: {data}")
print(f"最小元素: {data[0]}")

# 验证堆的性质
def verify_heap(heap):
    n = len(heap)
    for i in range(n):
        left = 2 * i + 1
        right = 2 * i + 2
        if left < n and heap[i] > heap[left]:
            return False
        if right < n and heap[i] > heap[right]:
            return False
    return True

print(f"是否满足堆性质: {verify_heap(data)}")

原始列表: [9, 3, 8, 1, 6, 2, 5, 4, 7]
转换为堆后: [1, 3, 2, 4, 6, 8, 5, 9, 7]
最小元素: 1
是否满足堆性质: True


## 6. nlargest() 和 nsmallest() - 找到最大/最小的 n 个元素

从可迭代对象中找到最大或最小的 n 个元素。

In [8]:
# 创建测试数据
numbers = [23, 7, 89, 12, 45, 67, 3, 56, 78, 1]
print(f"原始数据: {numbers}")

# 找到最大的 3 个元素
largest_3 = heapq.nlargest(3, numbers)
print(f"最大的 3 个元素: {largest_3}")

# 找到最小的 3 个元素
smallest_3 = heapq.nsmallest(3, numbers)
print(f"最小的 3 个元素: {smallest_3}")

# 使用 key 参数
students = [
    {'name': 'Alice', 'score': 85},
    {'name': 'Bob', 'score': 92},
    {'name': 'Charlie', 'score': 78},
    {'name': 'David', 'score': 95},
    {'name': 'Eve', 'score': 88}
]

# 找到分数最高的 2 个学生
top_2 = heapq.nlargest(2, students, key=lambda x: x['score'])
print(f"分数最高的 2 个学生: {top_2}")

# 找到分数最低的 2 个学生
bottom_2 = heapq.nsmallest(2, students, key=lambda x: x['score'])
print(f"分数最低的 2 个学生: {bottom_2}")

原始数据: [23, 7, 89, 12, 45, 67, 3, 56, 78, 1]
最大的 3 个元素: [89, 78, 67]
最小的 3 个元素: [1, 3, 7]
分数最高的 2 个学生: [{'name': 'David', 'score': 95}, {'name': 'Bob', 'score': 92}]
分数最低的 2 个学生: [{'name': 'Charlie', 'score': 78}, {'name': 'Alice', 'score': 85}]


## 7. 最大堆的实现

由于 heapq 只提供最小堆，实现最大堆需要一些技巧。

In [9]:
# 方法1: 使用负数
def max_heap_with_negation():
    max_heap = []
    
    # 添加元素时取负数
    for num in [3, 1, 4, 1, 5, 9, 2, 6]:
        heapq.heappush(max_heap, -num)
    
    print(f"最大堆(使用负数): {[-x for x in max_heap]}")
    
    # 弹出元素时再取负数
    result = []
    while max_heap:
        result.append(-heapq.heappop(max_heap))
    
    print(f"从大到小弹出: {result}")

max_heap_with_negation()

# 方法2: 自定义类包装
class MaxHeapItem:
    def __init__(self, value):
        self.value = value
    
    def __lt__(self, other):
        return self.value > other.value  # 反转比较
    
    def __repr__(self):
        return f"MaxHeapItem({self.value})"

def max_heap_with_wrapper():
    max_heap = []
    
    for num in [3, 1, 4, 1, 5, 9, 2, 6]:
        heapq.heappush(max_heap, MaxHeapItem(num))
    
    print(f"\n最大堆(使用包装类): {[item.value for item in max_heap]}")
    
    result = []
    while max_heap:
        result.append(heapq.heappop(max_heap).value)
    
    print(f"从大到小弹出: {result}")

max_heap_with_wrapper()

最大堆(使用负数): [9, 6, 5, 4, 1, 3, 2, 1]
从大到小弹出: [9, 6, 5, 4, 3, 2, 1, 1]

最大堆(使用包装类): [9, 6, 5, 4, 1, 3, 2, 1]
从大到小弹出: [9, 6, 5, 4, 3, 2, 1, 1]


## 8. 实际应用示例

### 示例1: 合并多个有序列表

In [10]:
def merge_sorted_lists(lists):
    """合并多个有序列表"""
    heap = []
    result = []
    
    # 将每个列表的第一个元素加入堆
    for i, lst in enumerate(lists):
        if lst:
            heapq.heappush(heap, (lst[0], i, 0))  # (值, 列表索引, 元素索引)
    
    while heap:
        val, list_idx, elem_idx = heapq.heappop(heap)
        result.append(val)
        
        # 如果当前列表还有更多元素，将下一个元素加入堆
        if elem_idx + 1 < len(lists[list_idx]):
            next_val = lists[list_idx][elem_idx + 1]
            heapq.heappush(heap, (next_val, list_idx, elem_idx + 1))
    
    return result

# 测试
sorted_lists = [
    [1, 4, 7],
    [2, 5, 8],
    [3, 6, 9, 10]
]

merged = merge_sorted_lists(sorted_lists)
print(f"原列表: {sorted_lists}")
print(f"合并结果: {merged}")

原列表: [[1, 4, 7], [2, 5, 8], [3, 6, 9, 10]]
合并结果: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


### 示例2: Top K 频繁元素

In [11]:
from collections import Counter

def top_k_frequent(nums, k):
    """找到数组中出现频率最高的 k 个元素"""
    # 统计频率
    counter = Counter(nums)
    
    # 使用堆找到频率最高的 k 个元素
    return heapq.nlargest(k, counter.keys(), key=counter.get)

# 测试
nums = [1, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4]
k = 2

result = top_k_frequent(nums, k)
counter = Counter(nums)

print(f"数组: {nums}")
print(f"频率统计: {dict(counter)}")
print(f"出现频率最高的 {k} 个元素: {result}")

数组: [1, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4]
频率统计: {1: 3, 2: 2, 3: 4, 4: 5}
出现频率最高的 2 个元素: [4, 3]


### 示例3: 滑动窗口中位数

In [12]:
class MedianFinder:
    """维护数据流中位数的类"""
    
    def __init__(self):
        self.small = []  # 最大堆，存储较小的一半
        self.large = []  # 最小堆，存储较大的一半
    
    def add_num(self, num):
        # 先加入最大堆（small），然后平衡
        heapq.heappush(self.small, -num)
        
        # 确保最大堆的最大值不超过最小堆的最小值
        if self.small and self.large and (-self.small[0]) > self.large[0]:
            val = -heapq.heappop(self.small)
            heapq.heappush(self.large, val)
        
        # 平衡两个堆的大小
        if len(self.small) > len(self.large) + 1:
            val = -heapq.heappop(self.small)
            heapq.heappush(self.large, val)
        
        if len(self.large) > len(self.small) + 1:
            val = heapq.heappop(self.large)
            heapq.heappush(self.small, -val)
    
    def find_median(self):
        if len(self.small) > len(self.large):
            return -self.small[0]
        elif len(self.large) > len(self.small):
            return self.large[0]
        else:
            return (-self.small[0] + self.large[0]) / 2

# 测试
mf = MedianFinder()
nums = [1, 2, 3, 4, 5]

for num in nums:
    mf.add_num(num)
    print(f"添加 {num} 后的中位数: {mf.find_median()}")

添加 1 后的中位数: 1
添加 2 后的中位数: 1.5
添加 3 后的中位数: 2
添加 4 后的中位数: 2.5
添加 5 后的中位数: 3


## 9. 性能对比

对比不同操作的性能特点。

In [13]:
import time

def performance_comparison():
    # 生成测试数据
    n = 10000
    data = [random.randint(1, 1000) for _ in range(n)]
    
    # 测试 heapify 性能
    data_copy = data.copy()
    start = time.time()
    heapq.heapify(data_copy)
    heapify_time = time.time() - start
    
    # 测试逐个插入性能
    heap = []
    start = time.time()
    for item in data:
        heapq.heappush(heap, item)
    push_time = time.time() - start
    
    # 测试 nsmallest 性能
    start = time.time()
    smallest_100 = heapq.nsmallest(100, data)
    nsmallest_time = time.time() - start
    
    # 测试排序然后切片的性能
    start = time.time()
    sorted_data = sorted(data)[:100]
    sort_time = time.time() - start
    
    print(f"数据规模: {n}")
    print(f"heapify 时间: {heapify_time:.4f}s")
    print(f"逐个插入时间: {push_time:.4f}s")
    print(f"nsmallest(100) 时间: {nsmallest_time:.4f}s")
    print(f"排序+切片时间: {sort_time:.4f}s")
    
    print(f"\n结果验证: {smallest_100[:5] == sorted_data[:5]}")

performance_comparison()

数据规模: 10000
heapify 时间: 0.0002s
逐个插入时间: 0.0003s
nsmallest(100) 时间: 0.0002s
排序+切片时间: 0.0007s

结果验证: True


## 总结

### heapq 模块的主要特点：
1. **最小堆实现**：堆顶始终是最小元素
2. **基于列表**：直接在 Python 列表上操作，节省内存
3. **高效操作**：插入和删除的时间复杂度为 O(log n)
4. **实用函数**：提供 nlargest、nsmallest 等便利函数

### 常见使用场景：
- 优先队列实现
- Top K 问题
- 合并多个有序序列
- 实时数据流中位数
- 任务调度
- 图算法中的最短路径（Dijkstra）

### 注意事项：
- heapq 只提供最小堆，实现最大堆需要技巧
- 堆的性质只保证父子节点关系，不保证同层节点的顺序
- 适合需要频繁获取最值的场景
- 对于一次性查找 Top K，直接排序可能更简单